In [1]:
import pandas as pd
pd.set_option("display.max_seq_items", None)  
pd.set_option("display.width", None)         

In [2]:
raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")

C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\2691490454.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/ret_sample_update.csv")


In [3]:
data = raw_data.copy()

In [4]:
# for 2005 to 2025, read pickle file get gvkeys into a dictionary and delete the dataframe to free up memory
gvkeys_dict = {}
for year in range(2005, 2026):
    text_data = pd.read_pickle(f"C:\\Users\\alexi\\OneDrive\\Documents\\école\\McGill-FIAM\\2025\\Hackathon-Final-2025\\DATA ASSET MANAGEMENT HACKATHON 2025 FINALS\\TEXT DATA US by YEAR\\{year}\\text_us_{year}.pkl")
    gvkeys_dict[year] = text_data['gvkey'].unique().tolist()
    del text_data

In [5]:
# For every gvkey in the dictionary, filter the data dataframe to only include rows with those gvkeys for the corresponding year
filtered_data_list = []
for year in range(2005, 2026):
    # data from that year contains only gvkeys in gvkeys_dict[year]
    filtered_data = data[(data['year'] == year) & (data['gvkey'].isin(gvkeys_dict[year]))]
    filtered_data_list.append(filtered_data)

# concatenate all filtered dataframes
filtered_data = pd.concat(filtered_data_list, ignore_index=True)
del filtered_data_list

In [6]:
# Keep columns: date, excntry, stock_ret, year, month, char_date, market_equity, be_me, ni_me, at_gr1, tangibility, at_be, debt_me, div12m_me, eqpo_me, eqnetis_at, debt_iss, ni_be, profit_sale, gp_at, turnover_126d
filtered_data = filtered_data[['date', 'gvkey', 'excntry', 'stock_ret', 'year', 'month', 'char_date', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d']]

In [7]:
joining_table = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/North America Company Name Merge by DataDate-GVKEY-IID.csv")
# rename datadate to date
joining_table = joining_table.rename(columns={"datadate": "date"})

In [8]:
# --- Keys côté filtered_data ---
# NOTE: char_date dans ret_sample_update est souvent un "trading month-end" (ex: 2021-01-29),
# alors que joining_table peut contenir la fin de mois calendrier (ex: 2021-01-31).
# Pour éviter de perdre des mois entiers au merge, on normalise les deux côtés à la FIN DE MOIS CALENDAIRE.
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')

filtered_data['char_date_key'] = (
    pd.to_datetime(filtered_data['char_date'].astype(str), format='%Y%m%d', errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')   # month-end calendrier
)

# --- Keys côté joining_table ---
joining_table['gvkey_key'] = pd.to_numeric(joining_table['gvkey'], errors='coerce').astype('Int64')
joining_table['char_date_key'] = (
    pd.to_datetime(joining_table['date'], errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')   # month-end calendrier
)

# (optionnel) éviter les duplications si plusieurs lignes par gvkey-date
joining_table = joining_table.drop_duplicates(subset=['gvkey_key', 'char_date_key'])

# --- Merge sur gvkey + char_date_key (month-end calendrier) ---
filtered_data = filtered_data.merge(
    joining_table[['gvkey_key', 'char_date_key', 'tic', 'conm']],
    on=['gvkey_key', 'char_date_key'],
    how='left'
)


In [9]:
sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")

C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\3118638869.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  sector_mapping = pd.read_csv("C:/Users/alexi/OneDrive/Documents/école/McGill-FIAM/2025/Hackathon-Final-2025/DATA ASSET MANAGEMENT HACKATHON 2025 FINALS/MAIN DATA and SUPPORTING CODES/Sector Info SIC and GIC codes All Countries to merge by GVKEY and Date.csv")


In [10]:
# --- Keys côté filtered_data ---
filtered_data['gvkey_key'] = pd.to_numeric(filtered_data['gvkey'], errors='coerce').astype('Int64')
filtered_data['date_key'] = (
    pd.to_datetime(filtered_data['date'].astype(str), format='%Y%m%d', errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')   # month-end calendrier
)

# --- Keys sector_mapping ---
# Le fichier SIC/GICS contient souvent 'date' (trading month-end) ET parfois 'eom' (calendar month-end).
# On privilégie 'eom' si présent, sinon on retombe sur 'date', puis on normalise en month-end calendrier.
cols = [c for c in ['gvkey', 'date', 'eom', 'gics', 'sic', 'naics'] if c in sector_mapping.columns]
sm = sector_mapping[cols].copy()

sm['gvkey_key'] = pd.to_numeric(sm['gvkey'], errors='coerce').astype('Int64')

date_col = 'eom' if 'eom' in sm.columns else 'date'
# Supporte int YYYYMMDD, string YYYYMMDD, ou YYYY-MM-DD
sm['_raw_date'] = sm[date_col].astype(str).str.replace('-', '').str.slice(0, 8)
sm['date_key'] = (
    pd.to_datetime(sm['_raw_date'], format='%Y%m%d', errors='coerce')
      .dt.to_period('M')
      .dt.to_timestamp('M')
)

sm = sm.drop(columns=['_raw_date'])
# éviter duplications
sm = sm.drop_duplicates(subset=['gvkey_key', 'date_key'])

# --- Merge ---
filtered_data = (
    filtered_data.merge(
        sm[['gvkey_key', 'date_key', 'gics', 'sic', 'naics']],
        on=['gvkey_key', 'date_key'],
        how='left'
    )
    .drop(columns=['gvkey_key', 'date_key'])
)


In [11]:
# 1) Extraire le code secteur GICS (2 premiers chiffres) à partir du code GICS 8 chiffres
# ex: 20101010 -> 20
filtered_data['gics_sector_code'] = (
    pd.to_numeric(filtered_data['gics'], errors='coerce')
      .floordiv(10**6)
      .astype('Int64')
)

# 2) Mapping MSCI / GICS (11 secteurs)
gics_sector_map = {
    10: "Energy",
    15: "Materials",
    20: "Industrials",
    25: "Consumer Discretionary",
    30: "Consumer Staples",
    35: "Health Care",
    40: "Financials",
    45: "Information Technology",
    50: "Communication Services",
    55: "Utilities",
    60: "Real Estate",
}

# 3) Ajouter le nom du secteur
filtered_data['gics_sector_name'] = filtered_data['gics_sector_code'].map(gics_sector_map)
filtered_data

,date,gvkey,excntry,stock_ret,year,month,char_date,market_equity,be_me,ni_me,...,gp_at,turnover_126d,char_date_key,tic,conm,gics,sic,naics,gics_sector_code,gics_sector_name
0,20050228,1243.0,CAN,0.006239,2005,2,20050131,14740.873131,0.806940,0.048776,...,0.132266,0.002816,2005-01-31,AL.1,ALCAN INC,15104010.0,3350.0,331319.0,15,Materials
1,20050228,2697.0,CAN,0.219169,2005,2,20050131,5361.886510,0.502860,0.061368,...,0.259079,0.003992,2005-01-31,NXY,NEXEN INC,10102020.0,1311.0,211111.0,10,Energy
2,20050228,3153.0,CAN,0.135651,2005,2,20050131,844.238381,0.204080,-0.045023,...,0.102018,NaN,2005-01-31,CDE,COEUR MINING INC,15104040.0,1044.0,212222.0,15,Materials
3,20050228,4040.0,CAN,0.004546,2005,2,20050131,7294.809288,0.595438,0.030264,...,0.191872,0.000004,2005-01-31,RRD,DONNELLEY (R R) & SONS CO,20201010.0,2750.0,32311.0,20,Industrials
4,20050228,4864.0,USA,0.207941,2005,2,20050131,602.468980,NaN,-0.449718,...,0.118811,0.038071,2005-01-31,FWLT,FOSTER WHEELER AG,20103010.0,1600.0,236210.0,20,Industrials
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636439,20250630,315318.0,USA,0.063407,2025,6,20250530,5184.735520,0.477729,0.046791,...,0.244835,0.010482,2025-05-31,ESI,ELEMENT SOLUTIONS INC,15101050.0,NaN,NaN,15,Materials
636440,20250630,316056.0,USA,0.013758,2025,6,20250530,12279.192300,0.122215,0.048660,...,0.397656,0.011089,2025-05-31,ALLE,ALLEGION PLC,20102010.0,NaN,NaN,20,Industrials
636441,20250630,317264.0,USA,0.138720,2025,6,20250530,913.093680,1.170207,0.176673,...,-0.067016,0.018638,2025-05-31,LPG,DORIAN LPG LTD,10102040.0,NaN,NaN,10,Energy
636442,20250630,326688.0,USA,0.113222,2025,6,20250530,10833.048800,0.321267,0.022228,...,0.201206,0.012945,2025-05-31,NVT,NVENT ELECTRIC PLC,20104010.0,NaN,NaN,20,Industrials


In [12]:
filtered_data['tic'].unique()

array(['AL.1', 'NXY', 'CDE', ..., 'BALY', 'USAR', 'FLYYQ'],
      shape=(5617,), dtype=object)

In [13]:
from pathlib import Path
import pandas as pd
import ast
import re

# Répertoire (relatif au notebook)
SP500_DIR = Path("sp500-master/sp500_constituants_2005_2024")

def norm_tic(s):
    """Normalise un ticker pour matcher les listes S&P500."""
    if pd.isna(s):
        return pd.NA
    s = str(s).strip().upper()
    # Harmoniser BRK.B vs BRK-B, BF.B vs BF-B, etc. (tes fichiers semblent utiliser '.')
    s = s.replace("-", ".")
    return s

def parse_ticker_list_cell(x):
    """Parse une cellule du type "['A', 'AAPL', ...]" -> list[str]."""
    if pd.isna(x):
        return []
    if isinstance(x, list):
        return [str(t) for t in x]
    s = str(x).strip()
    # Essaye literal_eval
    try:
        v = ast.literal_eval(s)
        if isinstance(v, list):
            return [str(t) for t in v]
    except Exception:
        pass
    # Fallback regex: tout ce qui ressemble à un token alpha-num/./-
    return re.findall(r"[A-Z0-9\.\-]+", s.upper())

# 1) Construire une table (year, tic_norm) à partir des fichiers annuels
members = []
for year in range(2005, 2025):  # tes fichiers sont 2005..2024
    fp = SP500_DIR / f"{year}-sp500-ticker-list.csv"
    if not fp.exists():
        continue
    dfy = pd.read_csv(fp)
    if 'tickers' not in dfy.columns:
        continue

    # On prend la dernière ligne (souvent la liste à la fin d'année), sinon concat toutes les lignes
    # Ici: concat toutes les lignes, puis unique.
    all_tickers = []
    for x in dfy['tickers'].dropna().tolist():
        all_tickers.extend(parse_ticker_list_cell(x))

    all_tickers = [norm_tic(t) for t in all_tickers]
    all_tickers = [t for t in all_tickers if pd.notna(t)]

    if all_tickers:
        members.append(pd.DataFrame({'year': year, 'tic_norm': pd.unique(all_tickers)}))

sp500_members = pd.concat(members, ignore_index=True).dropna().drop_duplicates()

# 2) Filtrer filtered_data par année (S&P500 de l'année correspondante)
fd = filtered_data.copy()
fd['tic_norm'] = fd['tic'].map(norm_tic)

filtered_data = (
    fd.merge(sp500_members, on=['year', 'tic_norm'], how='inner')
      .drop(columns=['tic_norm'])
)


C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\2176200038.py:55: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  members.append(pd.DataFrame({'year': year, 'tic_norm': pd.unique(all_tickers)}))
C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\2176200038.py:55: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  members.append(pd.DataFrame({'year': year, 'tic_norm': pd.unique(all_tickers)}))
C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\2176200038.py:55: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  members.append(pd.DataFrame({'year': year, 'tic_norm': pd.unique(all_tickers)}))
C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\2176200038.py:55: FutureWarning: unique with argum

In [14]:
# Diagnostic: vérifier la couverture mensuelle (utile pour détecter des mois entiers qui disparaissent)
month_counts = (filtered_data.groupby(["year", "month"]).size().reset_index(name="n_rows"))
display(month_counts[month_counts["year"].between(2020, 2021)].sort_values(["year","month"]))


,year,month,n_rows
179,2020,1,430
180,2020,2,430
181,2020,3,430
182,2020,4,430
183,2020,5,432
184,2020,6,432
185,2020,7,432
186,2020,8,432
187,2020,9,432
188,2020,10,432


In [15]:
# Forward fill values par gvkey, trié par date pour les colonnes: stock_ret, 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d', 'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name'
filtered_data = filtered_data.sort_values(by=['gvkey', 'date'])
cols_to_ffill = ['stock_ret', 'market_equity', 'be_me', 'ni_me', 'at_gr1', 'tangibility', 'at_be', 'debt_me', 'div12m_me', 'eqpo_me', 'eqnetis_at', 'dbnetis_at', 'ni_be', 'ebit_sale', 'gp_at', 'turnover_126d', 'gics', 'sic', 'naics', 'gics_sector_code', 'gics_sector_name']
filtered_data[cols_to_ffill] = (filtered_data.groupby('gvkey')[cols_to_ffill].ffill())

In [16]:
# Réduction de la taille des données
# Garder données entre 2021 et 2023 inclus
# Éliminer la moitié des tickers

filtered_data = filtered_data[(filtered_data['year'] >= 2021) & (filtered_data['year'] <= 2023)]
filtered_data = filtered_data.sort_values(by=['gvkey'])
unique_gvkeys = filtered_data['gvkey'].unique()
reduced_gvkeys = unique_gvkeys[::2]  # garder un gvkey sur deux
filtered_data = filtered_data[filtered_data['gvkey'].isin(reduced_gvkeys)]

In [17]:
# Afficher le nombre de tickers unique par année
for year in range(2005, 2026):
    n_unique_tickers = filtered_data[filtered_data['year'] == year]['gvkey'].nunique()
    print(f"Year {year}: {n_unique_tickers} unique tickers")

Year 2005: 0 unique tickers
Year 2006: 0 unique tickers
Year 2007: 0 unique tickers
Year 2008: 0 unique tickers
Year 2009: 0 unique tickers
Year 2010: 0 unique tickers
Year 2011: 0 unique tickers
Year 2012: 0 unique tickers
Year 2013: 0 unique tickers
Year 2014: 0 unique tickers
Year 2015: 0 unique tickers
Year 2016: 0 unique tickers
Year 2017: 0 unique tickers
Year 2018: 0 unique tickers
Year 2019: 0 unique tickers
Year 2020: 0 unique tickers
Year 2021: 213 unique tickers
Year 2022: 218 unique tickers
Year 2023: 220 unique tickers
Year 2024: 0 unique tickers
Year 2025: 0 unique tickers


In [18]:
# === Add latest available (10-K / 10-Q) text for each (gvkey, date) ===
# This attaches the most recent filing text (annual/quarterly) available as-of each month-end observation.
#
# Expected pickle schema includes: date (YYYYMMDD int), gvkey, file_type, mgmt, rf
# We build an "as-of" merge by gvkey and date.

import os
import sys
import numpy as np
import pandas as pd

# --- Pickle compatibility (some pickles reference numpy._core) ---
import numpy as _np
sys.modules.setdefault("numpy._core", _np.core)
sys.modules.setdefault("numpy._core._multiarray_umath", _np.core._multiarray_umath)
sys.modules.setdefault("numpy._core.multiarray", _np.core.multiarray)

TEXT_ROOT = r"C:\Users\alexi\OneDrive\Documents\école\McGill-FIAM\2025\Hackathon-Final-2025\DATA ASSET MANAGEMENT HACKATHON 2025 FINALS\TEXT DATA US by YEAR"

# Determine year span to load (include 1 extra year for "as-of" lookback)
min_y = int(pd.to_numeric(filtered_data["year"], errors="coerce").min())
max_y = int(pd.to_numeric(filtered_data["year"], errors="coerce").max())
start_y = max(2005, min_y - 1)
end_y = min(2025, max_y)

panel_gvkeys = pd.to_numeric(filtered_data["gvkey"], errors="coerce").dropna().unique()

# Parse panel dates
panel_date_dt = pd.to_datetime(filtered_data["date"].astype(str), format="%Y%m%d", errors="coerce")
panel_max_dt = panel_date_dt.max()

text_frames = []
for year in range(start_y, end_y + 1):
    fp = os.path.join(TEXT_ROOT, str(year), f"text_us_{year}.pkl")
    if not os.path.exists(fp):
        print(f"[WARN] Missing text file: {fp}")
        continue

    t = pd.read_pickle(fp)

    # Keep only needed columns if present
    keep = [c for c in ["date", "gvkey", "file_type", "mgmt", "rf"] if c in t.columns]
    t = t[keep].copy()

    # Basic cleaning
    t["gvkey"] = pd.to_numeric(t["gvkey"], errors="coerce")
    t = t.dropna(subset=["gvkey"])
    t = t[t["gvkey"].isin(panel_gvkeys)].copy()

    # report_date
    t["report_date"] = pd.to_datetime(t["date"].astype("Int64").astype(str), format="%Y%m%d", errors="coerce")
    t = t.dropna(subset=["report_date"])
    t = t[t["report_date"] <= panel_max_dt].copy()

    # Ensure text columns exist
    if "mgmt" not in t.columns:
        t["mgmt"] = ""
    if "rf" not in t.columns:
        t["rf"] = ""
    t["mgmt"] = t["mgmt"].fillna("").astype(str)
    t["rf"] = t["rf"].fillna("").astype(str)

    # Combine into one text field (keeps whichever parts are present)
    mg = t["mgmt"].str.strip()
    rf = t["rf"].str.strip()
    t["report_text"] = (mg + "\n\n" + rf).str.strip()
    t.loc[rf.eq(""), "report_text"] = mg
    t.loc[mg.eq(""), "report_text"] = rf
    t["report_text"] = t["report_text"].replace("", np.nan)
    t = t.dropna(subset=["report_text"])

    # Keep only filing types we care about (annual/quarterly)
    if "file_type" in t.columns:
        t["file_type"] = t["file_type"].astype(str)
        t = t[t["file_type"].isin(["10Q", "10K", "10KSB"])].copy()
    else:
        t["file_type"] = ""

    text_frames.append(t[["gvkey", "report_date", "file_type", "report_text"]])

text_df = pd.concat(text_frames, ignore_index=True) if text_frames else pd.DataFrame(
    columns=["gvkey", "report_date", "file_type", "report_text"]
)

# If multiple filings on same day, prefer annual over quarterly
type_rank = {"10Q": 1, "10K": 2, "10KSB": 2}
text_df["type_rank"] = text_df["file_type"].map(type_rank).fillna(0).astype(int)
text_df = text_df.sort_values(["gvkey", "report_date", "type_rank"])
text_df = text_df.drop_duplicates(["gvkey", "report_date"], keep="last")

# As-of merge: for each (gvkey, date) take latest report_date <= date
filtered_data["gvkey"] = pd.to_numeric(filtered_data["gvkey"], errors="coerce")
filtered_data["date_dt"] = pd.to_datetime(filtered_data["date"].astype(str), format="%Y%m%d", errors="coerce")

filtered_data = filtered_data.dropna(subset=["date_dt", "gvkey"]).sort_values(["date_dt", "gvkey"]).reset_index(drop=True)
text_df = text_df.sort_values(["report_date", "gvkey"])

filtered_data = pd.merge_asof(
    filtered_data,
    text_df[["gvkey", "report_date", "file_type", "report_text"]],
    left_on="date_dt",
    right_on="report_date",
    by="gvkey",
    direction="backward",
)

filtered_data = filtered_data.rename(
    columns={
        "report_date": "latest_report_date",
        "file_type": "latest_report_type",
        "report_text": "latest_report_text",
    }
)

filtered_data = filtered_data.drop(columns=["date_dt"], errors="ignore")

print("Added latest_report_date/latest_report_type/latest_report_text to filtered_data.")
print("Coverage:", filtered_data["latest_report_text"].notna().mean())


C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\641770237.py:15: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core._multiarray_umath.
  sys.modules.setdefault("numpy._core._multiarray_umath", _np.core._multiarray_umath)
C:\Users\alexi\AppData\Local\Temp\ipykernel_21812\641770237.py:16: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of n

Added latest_report_date/latest_report_type/latest_report_text to filtered_data.
Coverage: 0.9992406024553854


In [19]:
filtered_data.head()

,date,gvkey,excntry,stock_ret,year,month,char_date,market_equity,be_me,ni_me,...,tic,conm,gics,sic,naics,gics_sector_code,gics_sector_name,latest_report_date,latest_report_type,latest_report_text
0,20210129,1045.0,USA,0.088776,2021,1,20201231,9800.739885,0.031038,-0.353953,...,AAL,AMERICAN AIRLINES GROUP INC,20302010.0,4512.0,481111.0,20,Industrials,2020-10-22,10Q,Item 1A. Risk Factors \n 90 Item 1A. Risk Fact...
1,20210129,1078.0,USA,0.132889,2021,1,20201231,194055.911590,0.157573,0.015923,...,ABT,ABBOTT LABORATORIES,35101010.0,3845.0,334510.0,35,Health Care,2020-11-04,10Q,Item 2. Management s Discussion and Analysis o...
2,20210129,1209.0,USA,-0.023644,2021,1,20201231,60395.008050,0.208648,0.031748,...,APD,AIR PRODUCTS & CHEMICALS INC,15101040.0,2810.0,325120.0,15,Materials,2020-11-19,10K,ITEM 7. MANAGEMENT S DISCUSSION AND ANALYSIS O...
3,20210129,1300.0,USA,-0.081476,2021,1,20201231,149248.610060,0.131378,0.039183,...,HON,HONEYWELL INTERNATIONAL INC,20105010.0,9997.0,999977.0,20,Industrials,2020-10-30,10Q,Item 2. \n Management s Discussion and Analysi...
4,20210129,1380.0,USA,0.022542,2021,1,20201231,16210.595111,0.414174,-0.196600,...,HES,HESS CORP,10102020.0,1311.0,2111.0,10,Energy,2020-11-05,10Q,Item 1A. Risk Factors. Item 1A. Risk Factors i...


In [98]:
filtered_data.to_csv("yfinance/filtered_sp500_data.csv", index=False)

In [ ]:
filtered_data.head()

In [ ]:
filtered_data['date'].dtypes

In [ ]:
filtered_data.columns